In [1]:
#读取csv格式文件
import pandas as pd
path = './input/' #用户的路径环境
train = pd.read_csv(path+'train_set.csv')
test = pd.read_csv(path+'test_set.csv')
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [2]:
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [3]:
test.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,25318,51,housemaid,married,unknown,no,174,no,no,telephone,29,jul,308,3,-1,0,unknown
1,25319,32,management,married,tertiary,no,6059,yes,no,cellular,20,nov,110,2,-1,0,unknown
2,25320,60,retired,married,primary,no,0,no,no,telephone,30,jul,130,3,-1,0,unknown
3,25321,32,student,single,tertiary,no,64,no,no,cellular,30,jun,598,4,105,5,failure
4,25322,41,housemaid,married,secondary,no,0,yes,yes,cellular,15,jul,368,4,-1,0,unknown


In [4]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import lightgbm as lgb
test['y']=-1

cate_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']


feature1 = cate_features + num_features
data = pd.concat([train,test])

def feat_count(df, df_feature,fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].count()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_count" % ("_".join(fe[0]))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left")#.fillna(0)
    return df,df_count.columns[1]
for df in feature1:
    data = feat_count(data,data,['ID'],df)[0]

data.head()

def feature_count(data, features):
    feature_name = 'count'
    for i in features:
        feature_name += '_' + i
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name})
    data = data.merge(temp, 'left', on=features)
    return data,feature_name


ll=[]
for f in['campaign', 'contact','default','education','housing','job','loan','marital','poutcome','pdays','previous']:
    data,_=feature_count(data,['day','month',f])
    ll.append(_)
sparse_feature= ['campaign','contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature=['age', 'balance','day','duration','campaign','pdays','previous']

def get_new_columns(name,aggs):
    l=[]
    for k in aggs.keys():
        for agg in aggs[k]:
            if str(type(agg))=="<class 'function'>":
                l.append(name + '_' + k + '_' + 'other')
            else:
                l.append(name + '_' + k + '_' + agg)
    return l
for d in tqdm(sparse_feature):
    aggs={}
    for s in sparse_feature:
        aggs[s]=['count','nunique']
    for den in dense_feature:
        aggs[den]=['mean','max','min','std']
    aggs.pop(d)
    temp=data.groupby(d).agg(aggs).reset_index()
    temp.columns=[d]+get_new_columns(d,aggs)
    data=pd.merge(data,temp,on=d,how='left')


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:16<00:00,  2.76s/it]


In [5]:
df_train_columns=data.columns.tolist()

In [6]:
num_features2 = [i for i in data.columns if i not in cate_features if i not in ['ID','y','new_con']]

In [11]:
#ID这里好像没用'ID',
import numpy as np

from scipy.sparse import csr_matrix

from sklearn.model_selection import KFold
from tqdm import tqdm_notebook

num_features=num_features2


# one_hot_feature =['job', 'marital','education','default','housing','loan','contact','month','poutcome']

cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    data[i] = lbl.fit_transform(data[i].astype(str))



# def encode_count(df,column_name):
#     lbl = preprocessing.LabelEncoder()
#     lbl.fit(list(df[column_name].values))
#     df[column_name] = lbl.transform(list(df[column_name].values))
#     return df

data['new_con'] = data['job'].astype(str)
for i in ['marital', 'education', 'contact','month','poutcome']:
    data['new_con'] = data['new_con'].astype(str) + '_' + data[i].astype(str)
data['new_con'] = data['new_con'].apply(lambda x: ' '.join(x.split('_')))





# for feature in cat_col:
#     try:
#         data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
#     except:
#         data[feature] = LabelEncoder().fit_transform(data[feature])
        
vector_feature = ['new_con']


train=data[data.y!=-1]

# train, test, train_y, test_y = train_test_split(train,train_y,test_size=0.2, random_state=2018)
test=data[data.y==-1]
res=test[['ID']]
# test=test.drop(['ID'],axis=1)
train_x=train[num_features]
test_x=test[num_features]
enc = OneHotEncoder()
for feature in cat_col:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')


cv=CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')

# trainxx, vali_x, trainyy, vali_y = train_test_split(train_x,train_y,test_size=0.2, random_state=2018)
n_splits=5
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )

train_y=train['y']
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    clf.random_state = clf.random_state + 1
    train_x1 = train_x[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    clf.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += clf.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/10
res.to_csv('./basesubmission.csv', index=False)

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[1]	training's auc: 0.908369	valid_1's auc: 0.903809
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.91096	valid_1's auc: 0.905471
[3]	training's auc: 0.924319	valid_1's auc: 0.913205
[4]	training's auc: 0.925539	valid_1's auc: 0.914075
[5]	training's auc: 0.93226	valid_1's auc: 0.925805
[6]	training's auc: 0.937232	valid_1's auc: 0.930782
[7]	training's auc: 0.937722	valid_1's auc: 0.931974
[8]	training's auc: 0.937808	valid_1's auc: 0.932007
[9]	training's auc: 0.938081	valid_1's auc: 0.932551
[10]	training's auc: 0.938074	valid_1's auc: 0.932231
[11]	training's auc: 0.938792	valid_1's auc: 0.93187
[12]	training's auc: 0.938749	valid_1's auc: 0.932225
[13]	training's auc: 0.939081	valid_1's auc: 0.932197
[14]	training's auc: 0.939766	valid_1's auc: 0.932817
[15]	training's auc: 0.940216	valid_1's auc: 0.933579
[16]	training's auc: 0.941524	valid_1's auc: 0.934347
[17]	training's auc: 0.941493	valid_1's auc: 0.934386
[18]	training's auc: 0.941514	v

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[1]	training's auc: 0.920834	valid_1's auc: 0.894923
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.924211	valid_1's auc: 0.911594
[3]	training's auc: 0.917222	valid_1's auc: 0.905708
[4]	training's auc: 0.927272	valid_1's auc: 0.913442
[5]	training's auc: 0.924343	valid_1's auc: 0.910674
[6]	training's auc: 0.92918	valid_1's auc: 0.915471
[7]	training's auc: 0.933159	valid_1's auc: 0.91658
[8]	training's auc: 0.934736	valid_1's auc: 0.9178
[9]	training's auc: 0.938259	valid_1's auc: 0.924975
[10]	training's auc: 0.93922	valid_1's auc: 0.926285
[11]	training's auc: 0.939865	valid_1's auc: 0.926778
[12]	training's auc: 0.939435	valid_1's auc: 0.927982
[13]	training's auc: 0.939388	valid_1's auc: 0.92825
[14]	training's auc: 0.94032	valid_1's auc: 0.930004
[15]	training's auc: 0.941002	valid_1's auc: 0.930688
[16]	training's auc: 0.94152	valid_1's auc: 0.930552
[17]	training's auc: 0.94201	valid_1's auc: 0.931337
[18]	training's auc: 0.942749	valid_1

[300]	training's auc: 0.97015	valid_1's auc: 0.939823
[301]	training's auc: 0.970213	valid_1's auc: 0.939835
[302]	training's auc: 0.970284	valid_1's auc: 0.939829
[303]	training's auc: 0.970355	valid_1's auc: 0.939899
[304]	training's auc: 0.97042	valid_1's auc: 0.939876
[305]	training's auc: 0.970488	valid_1's auc: 0.939869
[306]	training's auc: 0.970543	valid_1's auc: 0.939894
[307]	training's auc: 0.970602	valid_1's auc: 0.939873
[308]	training's auc: 0.970672	valid_1's auc: 0.939838
[309]	training's auc: 0.97073	valid_1's auc: 0.939854
[310]	training's auc: 0.970786	valid_1's auc: 0.939845
[311]	training's auc: 0.970853	valid_1's auc: 0.939843
[312]	training's auc: 0.970924	valid_1's auc: 0.939873
[313]	training's auc: 0.970989	valid_1's auc: 0.939885
[314]	training's auc: 0.971069	valid_1's auc: 0.939897
[315]	training's auc: 0.971145	valid_1's auc: 0.939977
[316]	training's auc: 0.971208	valid_1's auc: 0.939964
[317]	training's auc: 0.97128	valid_1's auc: 0.939994
[318]	training

...

[173]	training's auc: 0.959925	valid_1's auc: 0.93923
[174]	training's auc: 0.960036	valid_1's auc: 0.939212
[175]	training's auc: 0.960125	valid_1's auc: 0.939235
[176]	training's auc: 0.960183	valid_1's auc: 0.93925
[177]	training's auc: 0.960289	valid_1's auc: 0.939219
[178]	training's auc: 0.960364	valid_1's auc: 0.939255
[179]	training's auc: 0.960458	valid_1's auc: 0.939232
[180]	training's auc: 0.960535	valid_1's auc: 0.939276
[181]	training's auc: 0.960634	valid_1's auc: 0.939303
[182]	training's auc: 0.960715	valid_1's auc: 0.93926
[183]	training's auc: 0.960804	valid_1's auc: 0.939271
[184]	training's auc: 0.960906	valid_1's auc: 0.939336
[185]	training's auc: 0.960988	valid_1's auc: 0.939406
[186]	training's auc: 0.961086	valid_1's auc: 0.93942
[187]	training's auc: 0.961188	valid_1's auc: 0.939489
[188]	training's auc: 0.961276	valid_1's auc: 0.939524
[189]	training's auc: 0.961381	valid_1's auc: 0.939502
[190]	training's auc: 0.961442	valid_1's auc: 0.939538
[191]	training

[471]	training's auc: 0.9793	valid_1's auc: 0.941418
[472]	training's auc: 0.97934	valid_1's auc: 0.941398
[473]	training's auc: 0.979389	valid_1's auc: 0.941364
[474]	training's auc: 0.979417	valid_1's auc: 0.941365
[475]	training's auc: 0.979475	valid_1's auc: 0.941362
[476]	training's auc: 0.979513	valid_1's auc: 0.941379
[477]	training's auc: 0.979553	valid_1's auc: 0.94136
[478]	training's auc: 0.979575	valid_1's auc: 0.941377
[479]	training's auc: 0.979612	valid_1's auc: 0.941364
[480]	training's auc: 0.97965	valid_1's auc: 0.941372
[481]	training's auc: 0.97969	valid_1's auc: 0.941374
[482]	training's auc: 0.979735	valid_1's auc: 0.941383
[483]	training's auc: 0.97977	valid_1's auc: 0.941372
[484]	training's auc: 0.979807	valid_1's auc: 0.941415
[485]	training's auc: 0.979847	valid_1's auc: 0.941407
[486]	training's auc: 0.979872	valid_1's auc: 0.941436
[487]	training's auc: 0.979917	valid_1's auc: 0.941451
[488]	training's auc: 0.979971	valid_1's auc: 0.941486
[489]	training's 

[220]	training's auc: 0.964962	valid_1's auc: 0.940515
[221]	training's auc: 0.965059	valid_1's auc: 0.940424
[222]	training's auc: 0.965164	valid_1's auc: 0.940464
[223]	training's auc: 0.965282	valid_1's auc: 0.940439
[224]	training's auc: 0.96536	valid_1's auc: 0.940502
[225]	training's auc: 0.965439	valid_1's auc: 0.94047
[226]	training's auc: 0.965528	valid_1's auc: 0.940529
[227]	training's auc: 0.965609	valid_1's auc: 0.940548
[228]	training's auc: 0.965674	valid_1's auc: 0.94059
[229]	training's auc: 0.965736	valid_1's auc: 0.9406
[230]	training's auc: 0.965798	valid_1's auc: 0.940595
[231]	training's auc: 0.965876	valid_1's auc: 0.94056
[232]	training's auc: 0.966037	valid_1's auc: 0.94055
[233]	training's auc: 0.966109	valid_1's auc: 0.940567
[234]	training's auc: 0.966175	valid_1's auc: 0.940569
[235]	training's auc: 0.966261	valid_1's auc: 0.940575
[236]	training's auc: 0.96634	valid_1's auc: 0.940574
[237]	training's auc: 0.966419	valid_1's auc: 0.940606
[238]	training's a

[59]	training's auc: 0.948725	valid_1's auc: 0.923799
[60]	training's auc: 0.948882	valid_1's auc: 0.923914
[61]	training's auc: 0.949112	valid_1's auc: 0.92397
[62]	training's auc: 0.94922	valid_1's auc: 0.924142
[63]	training's auc: 0.949483	valid_1's auc: 0.924256
[64]	training's auc: 0.949665	valid_1's auc: 0.924428
[65]	training's auc: 0.949814	valid_1's auc: 0.924536
[66]	training's auc: 0.949983	valid_1's auc: 0.924702
[67]	training's auc: 0.950115	valid_1's auc: 0.924789
[68]	training's auc: 0.950234	valid_1's auc: 0.924958
[69]	training's auc: 0.950443	valid_1's auc: 0.925283
[70]	training's auc: 0.950598	valid_1's auc: 0.925209
[71]	training's auc: 0.95071	valid_1's auc: 0.925154
[72]	training's auc: 0.950788	valid_1's auc: 0.925134
[73]	training's auc: 0.95088	valid_1's auc: 0.925071
[74]	training's auc: 0.95101	valid_1's auc: 0.925061
[75]	training's auc: 0.95118	valid_1's auc: 0.925094
[76]	training's auc: 0.9513	valid_1's auc: 0.925122
[77]	training's auc: 0.951397	valid_

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
